In [1]:
import pandas as pd 
import numpy as np 
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
import gc

/home/dashiell/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('../input/merged-train.csv')
test = pd.read_csv('../input/merged-test.csv')

In [3]:
train.dropna(axis=0, inplace=True) # temporary
#X_train, X_valid, y_train, y_valid = train_test_split(train.drop(['msno', 'is_churn'], axis=1), train['is_churn'], test_size=0.05)

X_train = train.drop(['msno','is_churn'], axis=1)
y_train = train['is_churn']

print(X_train.shape, len(y_train))
X_train.head()

train_meta = X_train.copy()
train_meta['fold_id'] = -999
train_meta['M1'] = np.nan
train_meta['M2'] = np.nan

(869896, 18) 869896


In [4]:
def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', metrics.log_loss(labels, preds)

In [11]:
lgb_params = {
    'num_leaves': 108,
    'boosting_type': 'gbdt',
    'objective':'binary',
    'metric':'binary_logloss',
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_freq': 1,
    'max_bin': 128,
    'max_depth': 10,
}

xgb_params = {
    'eta': 0.02, #use 0.002
    'max_depth': 7,
    'objective': 'binary:logistic',
    #'eval_metric': 'logloss',
    'silent': True,
}

#X_train['foldID'] = -1
fold_id = 0
kf = StratifiedKFold(n_splits=3, shuffle=True)

for train_index, test_index in kf.split(X_train, y_train):
    print("fold", fold_id)
    lgb_params['seed'] = fold_id; xgb_params['seed'] = fold_id
    
    ### lgbm
    print('training lgbm')
    #d_train = lgb.Dataset(X_train.iloc[train_index], label=y_train.iloc[train_index])
    #d_valid = lgb.Dataset(X_train.iloc[test_index], label=y_train.iloc[test_index], reference=d_train)

    #lgb_fitted = lgb.train(lgb_params, d_train, num_boost_round=100, valid_sets=[d_valid], early_stopping_rounds=10)
    #lgb_preds = lgb_fitted.predict(X_train.iloc[test_index])
    
    #train_meta.iloc[test_index, train_meta.columns.get_loc('M1')] = lgb_preds
    
    ### xgb
    print('training xgbm')
    watchlist = [(xgb.DMatrix(X_train.iloc[train_index],y_train.iloc[train_index]), 'train'), 
                 (xgb.DMatrix(X_train.iloc[test_index], y_train.iloc[test_index]), 'valid')]
    
    xgb_fitted = xgb.train(xgb_params, xgb.DMatrix(X_train.iloc[train_index], y_train.iloc[train_index]), 
                      150,  watchlist, 
                           #feval=xgb_score, 
                           maximize=False, verbose_eval=50, early_stopping_rounds=50) #use 1500
    xgb_preds = xgb_fitted.predict(xgb.DMatrix(X_train.iloc[test_index]))
    
    train_meta.iloc[test_index, train_meta.columns.get_loc('M2')] = xgb_preds
    
    train_meta.iloc[test_index, train_meta.columns.get_loc('fold_id')] = fold_id # test fold
    fold_id = fold_id + 1


fold 0
training lgbm
training xgbm
[0]	train-error:0.03383	valid-error:0.033763
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
[50]	train-error:0.032621	valid-error:0.032607
[100]	train-error:0.031583	valid-error:0.031783
fold 1
training lgbm
training xgbm
[0]	train-error:0.033882	valid-error:0.034166
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
[50]	train-error:0.032426	valid-error:0.033014
[100]	train-error:0.031595	valid-error:0.03239
fold 2
training lgbm
training xgbm
[0]	train-error:0.033897	valid-error:0.034542
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
[50]	train-error:0.0324	valid-error:0.033083
[100]	train-error:0.031707	valid-error:0.032338


In [ ]:
'''
### model stacking ###

#lgb_preds = lgb_fitted.predict(test.drop(['msno', 'is_churn'], axis=1)).clip(0.0000001, 0.999999)

submission = pd.DataFrame()
submission['msno'] = test['msno']
submission['is_churn'] = lgb_preds

submission.to_csv('../output/submission.csv', index=False)
'''

In [6]:
train_meta

,city,bd,registered_via,registration_init_time,expiration_date,total_order,plan_net_worth,mean_payment_each_transaction,total_actual_payment,cancel_times,...,last_listen_date,dist_last_listen_registration,dist_expiration_last_listen,listens_per_day,female,male,unknown_gender,fold_id,M1,M2
0,18.0,36.0,9.0,20050406.0,20170907.0,2,149,74.500000,149,0,...,20170208.0,119802.0,699.0,0.000217,1,0,0,-999,NaN,NaN
1,10.0,38.0,9.0,20050407.0,20170321.0,23,3309,150.347826,3458,2,...,20170215.0,119808.0,106.0,0.004349,0,1,0,-999,NaN,NaN
2,11.0,27.0,9.0,20051016.0,20170203.0,10,1492,149.200000,1492,1,...,20170131.0,119115.0,72.0,0.001990,1,0,0,2,NaN,0.974019
3,13.0,23.0,9.0,20051102.0,20170926.0,2,1788,894.000000,1788,0,...,20170221.0,119119.0,705.0,0.006170,1,0,0,0,NaN,0.359959
4,3.0,27.0,9.0,20051228.0,20170927.0,8,3576,447.000000,3576,0,...,20170228.0,119000.0,699.0,0.006370,0,1,0,1,NaN,0.077367
5,6.0,23.0,9.0,20060331.0,20170215.0,22,2384,149.000000,3278,1,...,20170228.0,109897.0,-13.0,0.005578,1,0,0,-999,NaN,NaN
6,13.0,29.0,9.0,20060406.0,20170208.0,11,1770,160.909091,1770,0,...,20170209.0,109803.0,-1.0,0.002104,1,0,0,-999,NaN,NaN
7,11.0,22.0,9.0,20060425.0,20170906.0,22,3222,153.227273,3371,1,...,20170225.0,109800.0,681.0,0.005974,0,1,0,3,NaN,0.067437
8,18.0,22.0,9.0,20060826.0,20170908.0,22,3683,174.227273,3833,0,...,20170228.0,109402.0,680.0,0.006307,1,0,0,2,NaN,0.073477
9,11.0,30.0,9.0,20061123.0,20170324.0,5,596,119.200000,596,0,...,20170123.0,109000.0,201.0,0.000477,1,0,0,1,NaN,0.281184


In [10]:
xgb_fitted.predict(xgb.DMatrix(X_train.iloc[test_index]))

array([ 0.30381414,  0.97398382,  0.08131972, ...,  0.13765679,
        0.02512872,  0.02512872], dtype=float32)